## Persistence
Persistence in Langgraph refers to the ability to save and restore the state of workflow overtime. 

In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

llm = ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-lite')

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details..


{'topic': 'pizza',
 'joke': 'Why did the pizza go to therapy?\n\nBecause it had too many *crust* issues!',
 'explanation': 'The joke is a pun, playing on the double meaning of the word "crust." Here\'s the breakdown:\n\n*   **The Setup:** The question establishes that the pizza is experiencing a problem serious enough to warrant therapy.\n*   **The Punchline:** "Because it had too many *crust* issues!"\n    *   **"Crust" as a literal food item:** This is the obvious meaning, referring to the edge of the pizza.\n    *   **"Crust" as a play on "trust":** This is the core of the pun. "Crust" sounds like "trust." The joke implies the pizza has "trust issues," a common problem people discuss in therapy. It suggests the pizza is struggling with emotional problems.\n\n**In essence, the joke is funny because it uses a word associated with pizza (crust) to create a wordplay that mimics a common human problem (trust issues), leading to an unexpected and humorous reason for why the pizza would ne

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy?\n\nBecause it had too many *crust* issues!', 'explanation': 'The joke is a pun, playing on the double meaning of the word "crust." Here\'s the breakdown:\n\n*   **The Setup:** The question establishes that the pizza is experiencing a problem serious enough to warrant therapy.\n*   **The Punchline:** "Because it had too many *crust* issues!"\n    *   **"Crust" as a literal food item:** This is the obvious meaning, referring to the edge of the pizza.\n    *   **"Crust" as a play on "trust":** This is the core of the pun. "Crust" sounds like "trust." The joke implies the pizza has "trust issues," a common problem people discuss in therapy. It suggests the pizza is struggling with emotional problems.\n\n**In essence, the joke is funny because it uses a word associated with pizza (crust) to create a wordplay that mimics a common human problem (trust issues), leading to an unexpected and humorous reason for why

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy?\n\nBecause it had too many *crust* issues!', 'explanation': 'The joke is a pun, playing on the double meaning of the word "crust." Here\'s the breakdown:\n\n*   **The Setup:** The question establishes that the pizza is experiencing a problem serious enough to warrant therapy.\n*   **The Punchline:** "Because it had too many *crust* issues!"\n    *   **"Crust" as a literal food item:** This is the obvious meaning, referring to the edge of the pizza.\n    *   **"Crust" as a play on "trust":** This is the core of the pun. "Crust" sounds like "trust." The joke implies the pizza has "trust issues," a common problem people discuss in therapy. It suggests the pizza is struggling with emotional problems.\n\n**In essence, the joke is funny because it uses a word associated with pizza (crust) to create a wordplay that mimics a common human problem (trust issues), leading to an unexpected and humorous reason for wh

In [11]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Res

{'topic': 'pasta',
 'joke': "Why did the pasta get sent to the principal's office? \n\nBecause it was *saucy*!",
 'explanation': 'Here\'s an explanation of the joke:\n\n*   **The Setup:** The joke sets up a scenario involving a school, implying the pasta has misbehaved.\n*   **The Punchline:** The punchline "Because it was saucy!" plays on two meanings of the word "saucy":\n    *   **Literal:** Pasta is often served with sauce.\n    *   **Figurative:** "Saucy" means bold, impudent, or disrespectful. This is the reason the pasta might be in trouble with the principal.\n*   **The Humor:** The humor comes from the pun. The joke uses the literal meaning of "saucy" (being covered in sauce) to create a surprising and funny connection to the figurative meaning of "saucy" (misbehaving). It\'s a silly, lighthearted play on words.'}

In [17]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': "Why did the pasta get sent to the principal's office? \n\nBecause it was *saucy*!", 'explanation': 'Here\'s an explanation of the joke:\n\n*   **The Setup:** The joke sets up a scenario involving a school, implying the pasta has misbehaved.\n*   **The Punchline:** The punchline "Because it was saucy!" plays on two meanings of the word "saucy":\n    *   **Literal:** Pasta is often served with sauce.\n    *   **Figurative:** "Saucy" means bold, impudent, or disrespectful. This is the reason the pasta might be in trouble with the principal.\n*   **The Humor:** The humor comes from the pun. The joke uses the literal meaning of "saucy" (being covered in sauce) to create a surprising and funny connection to the figurative meaning of "saucy" (misbehaving). It\'s a silly, lighthearted play on words.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0b806a-cc8d-6f61-8004-7c2bd23b0e2f'}}, metadata={'so

In [ ]:
list(workflow.get_state_history(config2)) 

[StateSnapshot(values={'topic': 'pasta', 'joke': "Why did the pasta get sent to the principal's office? \n\nBecause it was *saucy*!", 'explanation': 'Here\'s an explanation of the joke:\n\n*   **The Setup:** The joke sets up a scenario involving a school, implying the pasta has misbehaved.\n*   **The Punchline:** The punchline "Because it was saucy!" plays on two meanings of the word "saucy":\n    *   **Literal:** Pasta is often served with sauce.\n    *   **Figurative:** "Saucy" means bold, impudent, or disrespectful. This is the reason the pasta might be in trouble with the principal.\n*   **The Humor:** The humor comes from the pun. The joke uses the literal meaning of "saucy" (being covered in sauce) to create a surprising and funny connection to the figurative meaning of "saucy" (misbehaving). It\'s a silly, lighthearted play on words.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0b806a-cc8d-6f61-8004-7c2bd23b0e2f'}}, metadata={'s

### Time Travel
Useful for debugging

In [16]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0b8061-7f4a-6fa1-8002-a313f050bbd8"}})

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to therapy?\n\nBecause it had too many *crust* issues!', 'explanation': 'The joke is a pun, playing on the double meaning of the word "crust." Here\'s the breakdown:\n\n*   **The Setup:** The question establishes that the pizza is experiencing a problem serious enough to warrant therapy.\n*   **The Punchline:** "Because it had too many *crust* issues!"\n    *   **"Crust" as a literal food item:** This is the obvious meaning, referring to the edge of the pizza.\n    *   **"Crust" as a play on "trust":** This is the core of the pun. "Crust" sounds like "trust." The joke implies the pizza has "trust issues," a common problem people discuss in therapy. It suggests the pizza is struggling with emotional problems.\n\n**In essence, the joke is funny because it uses a word associated with pizza (crust) to create a wordplay that mimics a common human problem (trust issues), leading to an unexpected and humorous reason for why

In [20]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f0b8067-950b-62c7-8002-cef52bc5f653"}})

{'topic': 'pasta',
 'joke': "Why did the pasta break up with the sauce? \n\nBecause they just couldn't *noodle* their way through the relationship anymore!",
 'explanation': 'The joke plays on a pun, using a word that sounds similar to a relevant term. Here\'s a breakdown:\n\n*   **The Setup:** The question establishes a relationship problem, hinting at a separation.\n*   **The Punchline:** "Because they just couldn\'t *noodle* their way through the relationship anymore!"\n    *   **"Noodle"**: This refers to the type of pasta (like spaghetti, linguine, etc.) and is used as a verb.\n    *   **"Noodle their way"**: This phrase means to find a way to navigate or handle a situation, often with difficulty.\n    *   **The pun**: The joke uses the word "noodle" as a pun for "noodle" which is a type of pasta. The joke is based on the idea of the pasta literally not being able to find their way through their relationship with the sauce.\n\n**Why it\'s funny:**\n\n*   **Wordplay:** The humor co

In [112]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the mushroom go to the pizza party? Because he was a fungi and everyone wanted a pizza him!', 'explanation': 'This joke plays on the word "fun guy" (fungi) which sounds like "fungi," a type of mushroom. The play on words is that the mushroom went to the pizza party because he was a "fun guy" and people wanted to "pizza" (see) him. The joke is a pun that combines the idea of mushrooms being fungi with the concept of being a fun person at a party.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc70-100a-6bff-8002-7d6c3d37b1f4'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-29T21:57:21.959833+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc70-064c-630b-8001-707d60a085ad'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the mushroom go to the 

#### Updating State

In [113]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f06cc72-ca16-6359-8001-7eea05e07dd2'}}

In [114]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc72-ca16-6359-8001-7eea05e07dd2'}}, metadata={'source': 'update', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-29T21:58:35.155132+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='0f085bb0-c1e8-d9fd-fb15-c427126b7cd6', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the mushroom go to the pizza party? Because he was a fungi and everyone wanted a pizza him!', 'explanation': 'This joke plays on the word "fun guy" (fungi) which sounds like "fungi," a type of mushroom. The play on words is that the mushroom went to the pizza party because he was a "fun guy" and people 

In [115]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa bring a ladder to the party? \nBecause it wanted to be the best snack in the room and rise to the occasion!',
 'explanation': 'This joke plays on the double meaning of the word "rise." In one sense, "rise" means to physically move upwards, which is why the samosa brought a ladder to the party. However, in another sense, "rise" can also mean to perform well or excel, as in rising to the occasion. So, the samosa brought a ladder to symbolize its desire to physically rise above the other snacks at the party and also to metaphorically rise to the occasion by being the best snack in the room.'}

In [116]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa bring a ladder to the party? \nBecause it wanted to be the best snack in the room and rise to the occasion!', 'explanation': 'This joke plays on the double meaning of the word "rise." In one sense, "rise" means to physically move upwards, which is why the samosa brought a ladder to the party. However, in another sense, "rise" can also mean to perform well or excel, as in rising to the occasion. So, the samosa brought a ladder to symbolize its desire to physically rise above the other snacks at the party and also to metaphorically rise to the occasion by being the best snack in the room.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc75-4407-6195-8003-b08dcfd27511'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-29T21:59:41.628661+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoin

### Fault Tolerance

In [2]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [3]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [4]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [5]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))